This is an implemintation of the algorithms described in the [article](https://zkproof.org/2021/05/05/hashwires-range-proofs-from-hash-functions/) on Range Proofs with Hash functions.

We will explore the case where Alice wants to prove her age and uses the following approach to do so.

### Signing functionality

In [174]:
from sage.crypto.util import carmichael_lambda
from hashlib import sha256

In [175]:
prime1 = 7151
prime2 = 8147
assert is_prime(prime1)
assert is_prime(prime2)
group_s = prime1 * prime2

In [176]:
lamb = carmichael_lambda(group_s)
lambR = IntegerModRing(lamb)
nR = IntegerModRing(group_s)

In [177]:
def hash(m):
    hex_hash = sha256(str(m).encode("utf-8")).hexdigest()
    return int(hex_hash, 16)

## Naive Hashchain

In [178]:
import random

random.seed(0)

In [179]:
# We need two collision resistant hash functions H0 and H1
# As hash is collision resistant, by adding an extra letter we still obtain two collision resistant functions
# Which has an universal distribution
def h0(m):
    return hash(str(m) + "0")

# n represents amount of times on needs to apply H1 function 
def h1(m, n = 1):
    m_hash = hash(str(m) + "1")
    
    if n == 1:
         return m_hash
        
    return h1(m_hash, n - 1)
    

### Setup

In [180]:
class Gov:
    # Create a public / private key pair with RSA
    def __init__(self):
        while (1):
            self.prk = lambR.random_element()
            if (gcd(self.prk, lamb) == 1):
                break
                
        self.pbk = ~self.prk
        
        # Max year supported for the verification. 
        # This is universal for all poeple without regard to their age
        # After the year passes all proofs will stop working
        self.max_year = self.get_max_year() # We expect this service to run for max 480 years
        
    
    # Generate a commitment for a person that they were born on that year
    # Please note that we can create such commitment at any point of time
    def new_comm(self, year):
        seed = random.randint(0, 2 ^ 128)
        base_hash = h0(seed)
        k = self.max_year - year
        
        final_hash = h1(base_hash, k)
        
        return base_hash, self.sign(final_hash),
        
    def sign(self, m):
        return nR(hash(m)) ** self.prk
    
    @staticmethod
    def get_max_year():
        return 2500

    
    def get_pbk(self):
        return self.pbk

In [181]:
class Alice:
    
    # This is done during Alice birth 
    # Alice gets assigned 
    def __init__(self, gov, birth_year):
        
        self.birth_year = birth_year
        self.base_hash, self.final_sgn = gov.new_comm(birth_year)
        
        
    def gen_age_proof(self, age_to_prove):
        # Minimum birth year for anyone to be at least that age
        persons_age = cur_year - self.birth_year
        # How much older is the person compare to the minimum required age
        diff = persons_age - age_to_prove
        return h1(self.base_hash, diff), self.final_sgn      

In [182]:
class Carol:
    @staticmethod
    def check_sgn(pbk, m, sgn):
        assert(nR(hash(m)) == sgn ** pbk)
    
    
    @classmethod
    def check_age(cls, age_proof, sgn, required_age, gov_pbk):
        m = h1(age_proof, Gov.get_max_year() - (cur_year - required_age))
        cls.check_sgn(gov_pbk, m, sgn)

In [183]:
cur_year = 2022


In [184]:
gov = Gov()
pbk = gov.get_pbk()

In [185]:
alice = Alice(gov, 1999)

In [186]:
required_age = 18
proof, sgn = alice.gen_age_proof(required_age)
Carol.check_age(proof, sgn, required_age, pbk)

Obvious attack vector is for Alice to steal someones else base_hash and signature and thus prove that she is of suitable age. This could be prevented by verifying photo ID or other means.

Key idea here is that we are prooving the minimum age of a person, without revealing the exact age to Carol. This is done by providing proof for which the verification does not depend on the age of the participant but rather only on the minimum age allowed.

Addtional issue is that to generate and verify the proof we need to do k computations of hashes, where k is the range of allowed values.

### Proove that number from range [p, q] is within [n, m]

In [243]:
p = -100
q = 100

In [284]:
class Authority:
    # Create a public / private key pair with RSA
    def __init__(self):
        while (1):
            self.prk = lambR.random_element()
            if (gcd(self.prk, lamb) == 1):
                break
                
        self.pbk = ~self.prk
        
    
    # Generate a commitment for a point from two sides 
    def new_comm(self, point):
        
        # The right way would be to generate a commitment with different seed for each side
        seed = random.randint(0, 2 ^ 128)
        base_hash = h0(seed)
        
        final_hash_l = h1(base_hash + 12, point - p) 
        final_hash_r = h1(base_hash + 32, q - point)
        
        return base_hash, (self.sign(final_hash_l), self.sign(final_hash_r))
        
    def sign(self, m):
        return nR(hash(m)) ** self.prk
    

    
    def get_pbk(self):
        return self.pbk

In [285]:
class Alice:
    
    def __init__(self, authority, point):
        
        self.point = point
        self.base_hash, self.final_sgn = authority.new_comm(point)
        
        
    def gen_bound_proof(self, bounds_to_prove):
        n, m = bounds_to_prove
        
        l_proove = h1(self.base_hash + 12, self.point - n)
        r_proove = h1(self.base_hash + 32, m - self.point)
        return (l_proove, r_proove), self.final_sgn 

In [286]:
class Bob:
    @staticmethod
    def check_sgn(pbk, m, sgn):
        assert(nR(hash(m)) == sgn ** pbk)
    
    
    @classmethod
    def check_proof(cls, bounds_proof, sgn, bounds, auth_pbk):
        n, m = bounds

        m_l = h1(bounds_proof[0], n - p)
        m_r = h1(bounds_proof[1], q - m)
        
        
        cls.check_sgn(auth_pbk, m_l, sgn[0])
        cls.check_sgn(auth_pbk, m_r, sgn[1])




In [287]:
point = 4

In [288]:
auth = Authority()
alice = Alice(auth, point)

68520129217206913941947985549598382454743628632234308724489396662548981749129 7449406650852220865543587337789747210116871910736751713452792382726813760501
95653899511805700557570316767692772521842754956888427024184978927165736183497


In [289]:
bounds = (-20, 7)
proof, sgn = alice.gen_bound_proof(bounds)

In [290]:
Bob.check_proof(proof, sgn, bounds, auth.get_pbk())

68520129217206913941947985549598382454743628632234308724489396662548981749129
